# 015. Isolated Forest 를 이용한 NYC Taxi 탑승 정보 anomaly 검출

- 이상을 감지하는 최신 기술 중 하나는 Isolation Forest 입니다. 이 알고리즘은 anomaly data가 few and different 데이터 포인트라는 사실을 기반으로 합니다. 이러한 속성의 결과로 anomaly data는 isolation 이라는 메커니즘에 영향을 받기 쉽습니다.  

- 이 방법은 매우 유용하며 기존의 모든 방법과 근본적으로 다릅니다. 일반적으로 사용되는 기본 거리 및 밀도 측정 보다 이상을 감지하는 더 효과적이고 효율적인 수단으로 isolation을 사용합니다. 또한 이 방법은 선형 시간 복잡도가 낮고 메모리 요구 사항이 적은 알고리즘입니다. 데이터 세트의 크기에 관계없이 고정 크기의 작은 하위 샘플을 사용하여 적은 수의 트리로 우수한 성능의 모델을 구축합니다.

- IsolationForest는 feature를 무작위로 선택한 다음 선택한 feature의 최대값과 최소값 사이의 분할 값을 무작위로 선택하여 관측치를 isolate 합니다.

- 재귀 분할은 트리 구조로 나타낼 수 있으므로 샘플을 분리하는 데 필요한 분할 수는 루트 노드에서 종료 노드까지의 경로 길이와 동일합니다.

- 이러한 random tree forest에서 평균을 낸 경로 길이는 정상성 판단과 결정 기능의 척도입니다.

- 랜덤 파티셔닝은 anomaly에 대해 더 짧은 경로를 생성합니다. 따라서 random tree forest가 집합적으로 특정 샘플에 대해 더 짧은 경로 길이를 생성하면 anomaly일 가능성이 높습니다.

- 일반적인 기계 학습 방법은 학습하려는 패턴이 균형을 이룰 때 더 잘 작동하는 경향이 있습니다.


### 격리 포리스트의 작동 방식

- Isolation Forest 알고리즘은 feature를 무작위로 선택한 다음 선택한 feature의 최대값과 최소값 사이의 분할 값을 무작위로 선택하여 관측치를 분리합니다. 

- Isolation forest는 기본적으로 데이터셋을 의사결정나무(Decision Tree) 형태로 표현해 정상값을 분리하기 위해서는 의사결정나무를 깊숙하게 타고 내려가야 하고, 반대로 이상값은 의사결정나무 상단부에서 분리할 수 있다는 것을 이용합니다. 이 특성을 사용해 의사결정나무를 몇 회 타고 내려가야 분리되는가를 기준으로 정상과 이상을 분리합니다.

- 알고리즘이 분리를 구성하는 방법은 먼저 isolation tree 또는 random decision tree 를 만드는 것입니다. 그런 다음 sample을 분리하기 위한 path의 길이로 score 를 계산합니다.

- Isolation forest의 장점

    - 군집기반 이상탐지 알고리즘에 비해 계산량이 매우 적다   
    - 강건한(Robust)한 모델을 만들 수 있다

### Data download

- 2014년 7월 부터 2015년 1월까지의 New York 시의 taxi 탑승 횟수 정보를 30분 단위로 기록한 data 
- 요일별 / 시간별로 탑승횟수를 비교하여 이상치(교통량이 평소 보다 특별히 많거나 적은 날) 검출

- timestamp column 을 object --> datetime64 로 변경

## Feature Engineering
- 시간 간격이 30분 단위이면 다루기 어려우므로, 1 시간 단위로 resample

-  `timestamp`로 부터 `시간`, `요일`을 new feature로 생성하여  column 추가  

## EDA를 통해 시각적으로 Abnormal 한 날짜 검출

- 요일별 평균 탑승 횟수 시각화

- 시간대별 평균 탑승 횟수 시각화

- 전체 시계열 시각화

2014.11 부근에 큰 peak 가 보이고, 2015.2 이전에 매우 작은 값이 보인다.  
2014-11-02 는 뉴욕마라톤이 있어 관광객이 많았으며,  2015-01-27 에는 눈보라가 있어 교통이 마비 되었음.  

해당일자 기간을 확대하여 시각화

## Isolated Forest를 이용하여 이상치 검출

### 1 - contamination parameter 를 이용한 threshold control  
contamination parameter - dataset 내의 이상치 비율

outliers 검출. anomaly 는 -1. normal 은 1.

outlier는 `yes`로 표시.

### 2 - 임의의 threshold 설정 

scores - 입력 샘플의 anomaly score 입니다. 낮을수록 비정상적입니다. 음수 점수는 outlier를 나타내고 양수 점수는 inlier를 나타냅니다.

- decision_function - 평균 anomaly score 계산

contamination parameter 와 임의의 score로 설정한 threshold 결과를 조합하여 비교하며 domain knowlege를 이용하여 outlier(교통량이 평소보다 특별히 많거나 적은 날) 를 구분하면 된다.  

- 2014-07-04~07 : 독립기념일  
- 2014-09-06 : 노동절 행진  
- 2014-11-02 : 뉴욕 마라톤 대회  
- 2015-01-26~28 : 심한 눈보라

결론 - EDA 를 통해 manually 찾아낸 abnormal 한 일자와 Isolated Forest 를 이용하여 Machine Learning으로 찾아낸 일자가 거의 일치함